In [21]:
import pandas as pd
import pymsteams
from dotenv import load_dotenv
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


load_dotenv()


def load_data():

    df = pd.read_csv('../data/recent.csv')

    df.ds = pd.to_datetime(df.ds)

    forecast = pd.read_csv('../data/forecast.csv')

    forecast.ds = pd.to_datetime(forecast.ds)

    current = df.iloc[0]

    current_ds = df.head(1).iloc[0].ds

    return df, forecast, current, current_ds


df, forecast, current, current_ds = load_data()


alert_types = [
    # 'Total Inflow hrly',
    #  'Ambulances hrly',
    'Total Stretcher pts',
    'Triage hallway pts',
    'Triage hallway pts TBS',
    #  'Resus Pts',
    #  'Totalpts in PODs except Psych',
    # 'Green Pts TBS',
    # 'Yellow Pts TBS',
    # 'Orange Pts TBS',
    'Consults > 2h in PODS except IM',
    'Consult for IM >4h in PODS',
    'Plain films reqs > 2 h in PODs'
    'CTs reqs > 2 h in PODs',
    #  'Post POD (Family room)',
    #  'QTrack Patients TBS',
    #  'GARAGE patient TBS',
    'Consults > 2h in Vertical Except IM',
    'Consult for IM >4h in Vertical',
    'Plain films reqs > 2 hr in Vertical',
    'CTs reqs > 2 hrs in Vertical',
    'Total Pod TBS',
    'Total Vertical TBS']

alert_categories = {
    # 'Total Inflow hrly',
    #  'Ambulances hrly',
    'Total Stretcher pts': 'Patient Volume',
    'Triage hallway pts': 'Patient Volume',
    'Triage hallway pts TBS': 'Patient Volume',
    #  'Resus Pts',
    #  'Totalpts in PODs except Psych',
    # 'Green Pts TBS': 'Patient Volume',
    # 'Yellow Pts TBS',
    # 'Orange Pts TBS',
    'Consults > 2h in PODS except IM': 'Consultations',
    'Consult for IM >4h in PODS': 'Consultations',
    'Plain films reqs > 2 h in PODs': 'Radiology',
    'CTs reqs > 2 h in PODs': 'Radiology',
    #  'Post POD (Family room)',
    #  'QTrack Patients TBS',
    #  'GARAGE patient TBS',
    'Consults > 2h in Vertical Except IM': 'Consultations',
    'Consult for IM >4h in Vertical': 'Consultations',
    'Plain films reqs > 2 hr in Vertical': 'Radiology',
    'CTs reqs > 2 hrs in Vertical': 'Radiology',
    'Total Pod TBS': 'Patient Volume',
    'Total Vertical TBS': 'Patient Volume'}


current_forecast = forecast.set_index('ds').loc[current_ds]


In [9]:


alerts = []

for column in alert_types:
    try:
        if current[column] > current_forecast[column+'_yhat_upper']:
            alerts.append({'category': alert_categories[column], 'metric': column, 'value': current[column], 'yhat_upper': round(
                current_forecast[column+'_yhat_upper'], 1)})
    except:
        continue


if alerts:
    alerts_df = pd.DataFrame(alerts)

    active_alert_categories = alerts_df.category.unique().tolist()

    myTeamsMessage = pymsteams.connectorcard(os.environ.get('TEAMS_WEBHOOK'))

    # myTeamsMessage.title("Overcrowding alert")

    myTeamsMessage.text(' ')

    for category in active_alert_categories:

        myMessageSection = pymsteams.cardsection()

        myMessageSection.title(category)

        for i, row in alerts_df[alerts_df['category'] == category].iterrows():

            myMessageSection.addFact(row['metric'], str(row['value']))

        myTeamsMessage.addSection(myMessageSection)

    myTeamsMessage.printme()

    # myTeamsMessage.send()

else:
    print('No alerts')


No alerts


In [6]:
alert = alerts[0]
alert

{'category': 'Radiology',
 'metric': 'Plain films reqs > 2 hr in Vertical',
 'value': 3,
 'yhat_upper': 2.5}

In [10]:
metric = 'Total Vertical TBS'

In [11]:
fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])
fig.show()
fig.write_image("../images/fig1.png")


In [12]:
# data_canada = px.data.gapminder().query("country == 'Canada'")
fig = px.line(df.head(8), x='ds', y=metric)
fig.update_yaxes(range=[0, df.head(8)['Total Vertical TBS'].max()+5])
fig.show()
fig.write_image("../images/fig1.png")


In [24]:
def create_fig(df, forecast, metric):

    fig = make_subplots(
        specs=[[{"secondary_y": True}]], subplot_titles=[metric])


    fig.add_trace(go.Scatter(x=forecast.ds, y=forecast
                              [metric+'_yhat'], mode='lines', name='Total Vertical TBS (expected)', showlegend=False, line=dict(color='blue', width=1, dash='dot'),fill='tozeroy', fillcolor='lightgreen'))
    fig.add_trace(go.Scatter(x=forecast.ds, y=forecast
                              [metric+'_yhat_upper'], mode='lines', name='Total Vertical TBS (expected)', showlegend=False, line=dict(color='blue', width=1, dash='dot'), fill='tonexty', fillcolor='lightyellow'))
    fig.add_trace(go.Scatter(x=forecast.ds, y=forecast
                              [metric+'_yhat_upper']*10, mode='lines', name='Total Vertical TBS (expected)', showlegend=False, line=dict(color='blue', width=1, dash='dot'), fill='tonexty', fillcolor='lightpink'))
    fig.add_trace(go.Scatter(x=forecast.ds, y=forecast
                              [metric+'_yhat_lower'], mode='lines', name='Total Vertical TBS (expected)', showlegend=False, line=dict(color='blue', width=1, dash='dot')))

    fig.add_trace(go.Scatter(x=df.ds, y=df
                              [metric], mode='markers', name=metric, showlegend=False, line=dict(color='red', width=2)))
    

    
    # fig.add_trace(go.Scatter(x=verticalTBS_forecast.ds, y=verticalTBS_forecast.verticalTBS, mode='lines',
    #                           name='Total Inflow', showlegend=False,
    #                           line=dict(color='green', width=2, dash='dot')
    #                           ))

    # fig.update_yaxes(title_text="Total", secondary_y=False, range=[0, max(
    #     df[df.Date == current.Date]['Total Vertical TBS'].max(), todays_forecast['Total Vertical TBS_yhat_upper'].max())+1])
    # fig.show()
    fig.update_xaxes(range=[df.head(8).ds.min(), df.head(8).ds.max()])
    fig.update_yaxes(range=[0, max(df.head(8)[metric].max()*1.2, forecast.head(8)[metric+'_yhat_upper'].max()*1.2)])

    fig.write_image("../images/{}.png".format(metric))
    return fig
create_fig(df,forecast,'Total Vertical TBS')

In [14]:
forecast.head(8)

,ds,Stretcher Pts hrly_yhat,Stretcher Pts hrly_yhat_lower,Stretcher Pts hrly_yhat_upper,Stretcher Pts cum_yhat,Stretcher Pts cum_yhat_lower,Stretcher Pts cum_yhat_upper,Ambulatory Pts hrly_yhat,Ambulatory Pts hrly_yhat_lower,Ambulatory Pts hrly_yhat_upper,...,Psych pts waiting for admission_yhat_upper,Total Pod TBS_yhat,Total Pod TBS_yhat_lower,Total Pod TBS_yhat_upper,Total Vertical TBS_yhat,Total Vertical TBS_yhat_lower,Total Vertical TBS_yhat_upper,Stretcher Overflow_yhat,Stretcher Overflow_yhat_lower,Stretcher Overflow_yhat_upper
0,2023-02-23 02:00:00,3.418969,-1.242138,7.966199,21.815650,-11.384675,57.044902,1.484643,-2.988214,5.730416,...,16.408481,3.223806,-0.503948,6.791148,18.550559,6.877320,30.618279,3.243475,-1.627947,8.657441
1,2023-02-23 03:00:00,3.134827,-1.357636,7.793733,9.906050,-23.051796,42.189971,1.290169,-3.521691,5.914735,...,16.609178,2.808972,-0.698362,6.438995,17.186915,5.980622,29.306572,3.166739,-1.762101,7.992875
2,2023-02-23 04:00:00,2.933999,-1.769607,7.540554,16.545438,-16.436094,50.375716,0.957283,-3.349226,5.549052,...,16.554438,2.460375,-1.053693,5.948844,15.393905,3.501284,27.113308,3.148616,-2.326797,8.480181
3,2023-02-23 05:00:00,2.857679,-1.584774,7.300451,28.104105,-4.661399,61.596412,0.674116,-3.659927,4.872245,...,16.600104,2.223233,-1.468461,5.858159,13.986534,2.510853,25.728911,3.170779,-1.560839,8.117816
4,2023-02-23 06:00:00,3.103320,-1.405723,7.684959,32.989692,-1.525256,67.574876,0.915262,-3.067979,5.144990,...,16.770013,2.122048,-1.742372,5.607234,13.642796,2.088819,25.547805,3.215802,-1.869722,8.015915
5,2023-02-23 07:00:00,3.882899,-0.459944,8.350034,30.456641,-1.778928,64.466180,2.062028,-2.285576,6.842084,...,16.895851,2.165556,-1.384211,5.908704,14.393502,1.804528,26.524248,3.274546,-1.821891,8.315112
6,2023-02-23 11:00:00,9.575749,5.090610,14.145181,57.610989,22.836231,90.839983,8.054878,3.352307,12.625710,...,16.968096,3.639458,0.012698,7.109836,18.942336,7.490764,30.687260,3.752407,-1.000858,8.846721
7,2023-02-23 13:00:00,10.191752,5.327370,14.539552,67.962201,33.748760,104.117072,7.099823,2.674519,11.494207,...,15.768831,4.532566,0.856500,8.069716,22.888977,11.554220,34.653480,4.556502,-0.266222,9.539976


In [1]:
import pandas as pd

In [15]:
df = pd.read_csv('../data/since-2020.csv')
df.head(12)

,Date,Time,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,Total Inflow hrly,Total Inflow cum,Ambulances hrly,Ambulances cum,...,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS,Stretcher Overflow
0,03/02/23,9,6,30,7,19,13,49,1,11,...,1,0,0,3,19,16,2023-03-02 09:00:00,5,6,4
1,03/02/23,8,5,24,5,12,10,36,2,10,...,1,0,0,4,19,16,2023-03-02 08:00:00,5,13,4
2,03/02/23,7,3,19,2,7,5,26,2,8,...,4,0,3,4,19,14,2023-03-02 07:00:00,3,12,5
3,03/02/23,6,2,16,1,5,3,21,1,6,...,2,0,0,0,19,13,2023-03-02 06:00:00,1,14,5
4,03/02/23,5,1,14,1,4,2,18,1,5,...,2,0,0,0,19,13,2023-03-02 05:00:00,3,13,4
5,03/02/23,4,3,13,0,3,3,16,0,4,...,3,0,0,2,19,13,2023-03-02 04:00:00,3,15,1
6,03/02/23,3,0,10,0,3,0,13,0,4,...,3,0,0,1,19,13,2023-03-02 03:00:00,4,16,0
7,03/02/23,2,3,10,2,3,5,13,2,4,...,1,0,0,2,18,13,2023-03-02 02:00:00,8,17,0
8,03/02/23,1,6,6,2,2,8,8,2,2,...,1,0,0,2,17,13,2023-03-02 01:00:00,6,18,0
9,03/01/23,24,2,151,3,91,5,242,1,37,...,1,0,0,1,17,13,2023-03-02 00:00:00,4,15,0


In [25]:
from deta import Deta
from dotenv import load_dotenv

load_dotenv()

deta = Deta(os.environ.get("DETA_PROJECT_KEY"))

data = deta.Drive("data")

In [29]:
data = deta.Drive("data")


In [30]:
data.list()

{'names': []}

In [31]:
import pandas as pd
import tabula
import datetime

In [32]:
columns = [
    "Date",
    "Time",
    "Stretcher Pts hrly",
    "Stretcher Pts cum",
    "Ambulatory Pts hrly",
    "Ambulatory Pts cum",
    "Total Inflow hrly",
    "Total Inflow cum",
    "Ambulances hrly",
    "Ambulances cum",
    "FLS hrly",
    "Adm. requests cum",
    "Admissions cum",
    "Pts.waiting for admission CUM",
    "Triage hallway pts TBS",
    "Re-Oriented Nurse cum",
    "Re-Oriented MD QTrack D/C",
    "Re-Oriented MD QTrack NotD/C",
    "Resus Pts",
    "Totalpts in PODs except Psych",
    "Green Pts",
    "Green Pts TBS",
    "Yellow PTS",
    "Yellow Pts TBS",
    "Orancge Pts except psych",
    "Orange Pts TBS",
    "Consults > 2h in PODS except IM",
    "Consult for IM >4h in PODS",
    "Plain films reqs > 2 h in PODs",
    "CTs reqs > 2 h in PODs",
    "Post POD (Family room)",
    "Stretcher Pts in Vertical",
    "Stretcher Pts TBS in Vertical",
    "Stretcher Pts in Vertical on Lazyboy",
    "Vertical Pts Waiting for Results",
    "Ambulatory Pts in Vertical",
    "Ambulatory Pts TBS in Vertical",
    "QTrack Patients TBS",
    "GARAGE patient TBS",
    "Consults > 2h in Vertical Except IM",
    "Consult for IM >4h in Vertical",
    "Plain films reqs > 2 hr in Vertical",
    "CTs reqs > 2 hrs in Vertical",
    "Psych Stretcher Pts1pt",
    "Psych pts waiting for admission",
]

url = "https://www.dropbox.com/s/ckijmipu33z3feg/HourlyReport.pdf?dl=1"
output = []
first_row = tabula.read_pdf(
    url, pages=1, area=[[200, 6, 206, 1002]], silent=True)[0]
first_row = first_row.columns.tolist()
first_row_dict = dict(zip(columns, first_row))
output.append(first_row_dict)
first_row_date = first_row_dict['Date']
for row_start in [211, 223, 235, 247, 259, 271, 283]:
    try:
        new_row = tabula.read_pdf(
            url, pages=1, area=[[row_start, 0, row_start+7, 1002]], silent=True)[0]
        new_row = new_row.columns.tolist()
        new_row.insert(0, first_row_date)
        new_row_dict = dict(zip(columns, new_row))
        output.append(new_row_dict)
    except:
        break

output_df = pd.DataFrame(output)

missing_columns = []
first_row = tabula.read_pdf(
    url, pages=1, area=[[200, 308, 215, 341]], silent=True)[0]
first_row = first_row.columns.tolist()
first_row_dict = dict(
    zip(['Total Stretcher pts', 'Triage hallway pts'], first_row))
missing_columns.append(first_row_dict)
for row_start in [212, 224, 236, 248, 260, 272, 284]:
    try:
        new_row = tabula.read_pdf(
            url, pages=1, area=[[row_start, 308, row_start+8, 341]], silent=True)[0]
        new_row = new_row.columns.tolist()
        new_row_dict = dict(
            zip(['Total Stretcher pts', 'Triage hallway pts'], new_row))
        missing_columns.append(new_row_dict)
    except:
        break
missing_columns_df = pd.DataFrame(missing_columns)

output_df = pd.concat([output_df, missing_columns_df], axis=1)
output_df = output_df.dropna()

newColumns = [
    "Date",
    "Time",
    "Stretcher Pts hrly",
    "Stretcher Pts cum",
    "Ambulatory Pts hrly",
    "Ambulatory Pts cum",
    "Total Inflow hrly",
    "Total Inflow cum",
    "Ambulances hrly",
    "Ambulances cum",
    "FLS hrly",
    "Adm. requests cum",
    "Admissions cum",
    "Pts.waiting for admission CUM",
    "Total Stretcher pts",
    "Triage hallway pts",
    "Triage hallway pts TBS",
    "Re-Oriented Nurse cum",
    "Re-Oriented MD QTrack D/C",
    "Re-Oriented MD QTrack NotD/C",
    "Resus Pts",
    "Totalpts in PODs except Psych",
    "Green Pts",
    "Green Pts TBS",
    "Yellow PTS",
    "Yellow Pts TBS",
    "Orancge Pts except psych",
    "Orange Pts TBS",
    "Consults > 2h in PODS except IM",
    "Consult for IM >4h in PODS",
    "Plain films reqs > 2 h in PODs",
    "CTs reqs > 2 h in PODs",
    "Post POD (Family room)",
    "Stretcher Pts in Vertical",
    "Stretcher Pts TBS in Vertical",
    "Stretcher Pts in Vertical on Lazyboy",
    "Vertical Pts Waiting for Results",
    "Ambulatory Pts in Vertical",
    "Ambulatory Pts TBS in Vertical",
    "QTrack Patients TBS",
    "GARAGE patient TBS",
    "Consults > 2h in Vertical Except IM",
    "Consult for IM >4h in Vertical",
    "Plain films reqs > 2 hr in Vertical",
    "CTs reqs > 2 hrs in Vertical",
    "Psych Stretcher Pts1pt",
    "Psych pts waiting for admission",
]
output_df = output_df[newColumns]
for column in output_df.columns.tolist():
    if column in ['Date']:
        continue
    output_df[column] = output_df[column].astype('float').astype('int')

output_df["ds"] = pd.to_datetime(
    output_df["Date"] + " " + (output_df["Time"] - 1).astype(str) + ":00") + datetime.timedelta(hours=1)

output_df['Total Pod TBS'] = output_df['Green Pts TBS'] + \
    output_df['Yellow Pts TBS']+output_df['Orange Pts TBS'] + \
    output_df["Triage hallway pts TBS"]
output_df['Total Vertical TBS'] = output_df['Stretcher Pts TBS in Vertical'] + \
    output_df['Ambulatory Pts TBS in Vertical'] + \
    output_df['QTrack Patients TBS']+output_df['GARAGE patient TBS']
output_df['Stretcher Overflow'] = output_df["Triage hallway pts TBS"] + \
    output_df["Post POD (Family room)"]

In [33]:
output_df

,Date,Time,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,Total Inflow hrly,Total Inflow cum,Ambulances hrly,Ambulances cum,...,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS,Stretcher Overflow
0,03/02/23,10,6,40,9,24,15,64,1,13,...,2,0,0,5,19,16,2023-03-02 10:00:00,4,10,4
1,03/02/23,9,8,34,5,15,13,49,2,12,...,1,0,0,3,19,16,2023-03-02 09:00:00,5,6,4
2,03/02/23,8,6,26,4,10,10,36,2,10,...,1,0,0,4,19,16,2023-03-02 08:00:00,5,13,4
3,03/02/23,7,4,20,1,6,5,26,2,8,...,4,0,3,4,19,14,2023-03-02 07:00:00,3,12,5
4,03/02/23,6,2,16,1,5,3,21,1,6,...,2,0,0,0,19,13,2023-03-02 06:00:00,1,14,5
5,03/02/23,5,1,14,1,4,2,18,1,5,...,2,0,0,0,19,13,2023-03-02 05:00:00,3,13,4
6,03/02/23,4,3,13,0,3,3,16,0,4,...,3,0,0,2,19,13,2023-03-02 04:00:00,3,15,1
7,03/02/23,3,0,10,0,3,0,13,0,4,...,3,0,0,1,19,13,2023-03-02 03:00:00,4,16,0
